In [1]:
%reload_ext autoreload
import re
import os
import sys
import requests
import threading
import numpy as np
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict, defaultdict
pd.options.mode.chained_assignment = None  # default='warn'

ROOT = '/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/'

def add_helpers():
  '''
  add_helper mounts google drive and adds
  helper functions to the sys.path
  '''

  # if running on juypter/google colab, mount to google drive
  if 'google.colab' in str(get_ipython()): 
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/

  helper_dir_path = ROOT + 'helper/'
  print('\nHelpers:')
  pprint(sorted(os.listdir(helper_dir_path)))
  sys.path.append(helper_dir_path) # set to path of notebook

add_helpers()

Mounted at /content/drive
/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms

Helpers:
['Regions.py',
 'TeamNames.py',
 '__pycache__',
 'bettingLinesScraper.py',
 'gameLogScraper.py',
 'meta_info_scraper.py',
 'player_info_scraper.py',
 'player_table_scraper.py',
 'seasonScraper.py',
 'teamsScraper.py']


In [3]:
from player_info_scraper import player_info_scraper
from player_scraper import single_player_scraper

mj_meta, mj_data = single_player_scraper()
mj_data

Player Name: Michael Jordan
Searching for name in: https://www.basketball-reference.com/players/j
Michael Jordan found. DataFrames generated


,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,fg2_per_g,fg2a_per_g,fg2_pct,efg_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,mp,fg,fga,fg3,fg3a,fg2,fg2a,...,fg3_per_poss,fg3a_per_poss,fg2_per_poss,fg2a_per_poss,ft_per_poss,fta_per_poss,orb_per_poss,drb_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,,off_rtg,def_rtg,_blank,_basic,_league,_adj,_added,info,header_pos_estimates,header_plus_minus,header_turnovers,header_fouls,header_drawn,header_misc2,zzz,pct_fga,fg_pct_dist,pct_ast,dunks,corner3,heaves,college_id,team_name,salary
0,game,season,Michael Jordan,1984-85,21,CHI,NBA,SG,82,82,38.3,10.2,19.8,.515,0.1,0.6,.173,10.1,19.2,.526,.518,7.7,9.1,.845,2.0,4.5,6.5,5.9,2.4,0.8,3.5,3.5,28.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,season,Michael Jordan,1985-86,22,CHI,NBA,SG,18,7,25.1,8.3,18.2,.457,0.2,1.0,.167,8.2,17.2,.474,.462,5.8,6.9,.840,1.3,2.3,3.6,2.9,2.1,1.2,2.5,2.6,22.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,game,season,Michael Jordan,1986-87,23,CHI,NBA,SG,82,82,40.0,13.4,27.8,.482,0.1,0.8,.182,13.2,27.0,.491,.484,10.2,11.9,.857,2.0,3.2,5.2,4.6,2.9,1.5,3.3,2.9,37.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,game,season,Michael Jordan,1987-88,24,CHI,NBA,SG,82,82,40.4,13.0,24.4,.535,0.1,0.6,.132,13.0,23.7,.546,.537,8.8,10.5,.841,1.7,3.8,5.5,5.9,3.2,1.6,3.1,3.3,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,game,season,Michael Jordan,1988-89,25,CHI,NBA,SG,81,81,40.2,11.9,22.2,.538,0.3,1.2,.276,11.6,21.0,.553,.546,8.3,9.8,.850,1.8,6.2,8.0,8.0,2.9,0.8,3.6,3.0,32.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,all_salaries,season,Michael Jordan,1996-97,NaN,NaN,NBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Bulls,"$30,140,000"
211,all_salaries,season,Michael Jordan,1997-98,NaN,NaN,NBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Bulls,"$33,140,000"
212,all_salaries,season,Michael Jordan,2001-02,NaN,NaN,NBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Wizards,"$1,000,000"
213,all_salaries,season,Michael Jor

In [4]:
mj_vorp = mj_data[mj_data.vorp.notnull()] # drop vorp = NaN rows
mj_vorp = mj_vorp.dropna(how='all', axis='columns') # drop blank columns 
mj_vorp = mj_vorp.loc[mj_vorp['season'] != 'Career'] # only single-season statistics
mj_vorp[mj_vorp['vorp'] == max(mj_vorp['vorp'])] # max vorp season

,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,mp,DUMMY,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp
70,,season,Michael Jordan,1995-96,32,CHI,NBA,SG,82,3090,,29.4,.582,.141,.355,5.6,14.9,10.2,21.2,3.1,1.0,8.4,33.3,14.2,6.2,20.4,.317,8.3,2.2,10.5,9.8


In [5]:
mj_tables = mj_data['data_type']

In [7]:
mj_tables.unique()

array(['game', 'per_game', '', 'totals', 'advanced', 'per_minute',
       'per_poss', 'adjooting', 'pbp', 'shooting', 'all_star',
       'all_college_stats', 'all_salaries'], dtype=object)

In [11]:
mj_data[mj_data['data_type'] == 'advanced'].dropna(how='all', axis='columns')

,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,mp,DUMMY,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp
76,advanced,season,Michael Jordan,1984-85,21,CHI,NBA,SG,4,171,,24.7,.565,.103,.744,4.7,12.6,8.4,31.3,3.2,1.5,12.7,28.7,0.6,0.1,0.7,.198,7.6,1.9,9.5,0.5
77,advanced,season,Michael Jordan,1985-86,22,CHI,NBA,SG,3,135,,30.1,.584,.011,.411,4.0,13.6,8.4,26.5,2.5,1.9,11.1,39.2,0.4,0.0,0.5,.161,10.1,1.7,11.9,0.4
78,advanced,season,Michael Jordan,1986-87,23,CHI,NBA,SG,3,128,,28.1,.529,.060,.464,6.3,15.0,10.2,30.4,2.5,3.4,7.3,38.8,0.3,0.1,0.4,.165,9.9,2.9,12.7,0.5
79,advanced,season,Michael Jordan,1987-88,24,CHI,NBA,SG,10,427,,28.4,.598,.012,.381,5.8,12.7,9.2,23.1,2.9,1.5,11.4,35.2,1.3,0.8,2.1,.234,8.5,3.7,12.2,1.5
80,advanced,season,Michael Jordan,1988-89,25,CHI,NBA,SG,17,718,,29.9,.602,.090,.587,4.4,14.4,9.6,38.0,3.1,1.2,12.2,35.4,2.8,1.2,4.0,.270,9.1,3.1,12.1,2.5
81,advanced,season,Michael Jordan,1989-90,26,CHI,NBA,SG,16,674,,31.7,.592,.117,.373,4.2,16.6,10.2,34.5,3.5,1.5,10.1,36.1,2.7,1.3,4.0,.284,10.2,3.5,13.7,2.7
82,advanced,season,Michael Jordan,1990-91,27,CHI,NBA,SG,17,689,,32.0,.600,.069,.394,3.4,16.0,9.9,36.7,3.2,2.4,8.9,32.7,3.2,1.5,4.8,.333,10.5,4.1,14.6,2.9
83,advanced,season,Michael Jordan,1991-92,28,CHI,NBA,SG,22,920,,27.2,.571,.076,.325,4.9,13.2,9.1,28.3,2.6,1.2,10.9,37.1,2.4,1.7,4.1,.216,8.3,1.6,9.9,2.8
84,advanced,season,Michael Jordan,1992-93,29,CHI,NBA,SG,19,783,,30.1,.553,.136,.320,5.0,14.9,10.0,29.4,2.7,1.6,7.0,38.0,3.3,1.1,4.4,.270,9.7,1.8,11.6,2.7
85,advanced,season,Michael Jordan,1994-95,31,CHI,NBA,SG,10,420,,24.8,.557,.121,.319,6.0,12.7,9.5,22.5,3.0,2.8,12.7,35.7,0.7,0.6,1.3,.150,5.8,2.2,8.0,1.0
